# Meta Learning Cache Replacement Policy

## Decompress tar.gz

**Don't run unless you have a lot of storage!**

In [ ]:
import argparse
import os


EXTENSION = '.tar.gz'
BLKPARSE = '.blkparse'


def main(input_dir, output_dir):
  assert os.path.isdir(input_dir), "The input directory {} does not exist".format(input_dir)

  input_dir = input_dir.rstrip("/")
  if not os.path.isdir(output_dir):
    print("Creating output directory {}".format(output_dir))
    os.mkdir(output_dir)

  for file in os.listdir(input_dir):
    if not file.endswith(EXTENSION):
      continue
    
    print("Extracting and moving {}".format(file))
    execute_command("tar -xvzf {}".format(input_dir + '/' + file))
    execute_command("mv ./*{} {}".format(BLKPARSE, output_dir))
    

def execute_command(command):
  os.system(command)



In [ ]:
input_dir = "FIU_raw"
output_dir = "FIU_trace"
main(input_dir, output_dir)

## Install Dependency

In [1]:
import sys
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
from collections import Counter, deque, defaultdict
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
import pickle


## Block Cache Model

### Global Variable

In [3]:
# Maximum block number
maxpos = 1000000000000

# Number of features (Recency, Frequency, Block No.)
num_params = 3

# Cache Size
cache_size = 100

# Sequence Length
sequence_length = 5

# Storage
sequence_data = np.zeros((sequence_length, cache_size, num_params))

# How often to store the data (Removed currently)
sampling_freq = cache_size

# How many % of cache to use
eviction = int(0.7 * cache_size)  

# Results
lruCorrect = 0
lruIncorrect = 0

lfuCorrect = 0
lfuIncorrect = 0

# Variables
X = np.array([], dtype=np.int64).reshape(0,num_params)
Y = np.array([], dtype=np.int64).reshape(0,1)

### Load workload
**cheetah.cs.fiu.edu-110108-113008.1.blkparse** does not contain correct data.

In [4]:
train = "FIU_trace/cheetah.cs.fiu.edu-110108-113008.2.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
#trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.001)]
trainBlockTrace = trainBlockTrace[:200]
len(trainBlockTrace)

200

### Data Preprocessing and Data Construction

In [5]:
# Taken from Shehbaz
def get_recency(lru, cache):
    recency = []
    recency_dict = defaultdict(int)
    
    # Compute the recency order of each page in cache
    for time in range(len(lru)):
        recency_dict[lru[time]] = time
        
    for block in cache:
        recency.append(recency_dict[block])

    return recency

def get_frequency(lfu, cache):
    frequency = []
    
    for block in cache:
        frequency.append(lfu[block])
    return frequency

def normalize_columns(input):
    return normalize(input, axis=0)

In [32]:
def get_single_length_input(lfu, lru, cache, preprocess_func, cache_size):
    input_recency = get_recency(lru, cache)
    input_frequency = get_frequency(lfu, cache)
    input_block_num = cache[:]
    
    if len(input_recency) < cache_size:
        for i in range(0, cache_size - len(input_recency)):
            input_recency.append(0)
            input_frequency.append(0)
            input_block_num.append(0)
    
    # Columns: recency, frequency, block number
    # Row: cache location
    raw_input = np.column_stack((input_recency, input_frequency, input_block_num))
    
    return preprocess_func(raw_input)


SEQ_DIM = 0
def get_multiple_length_input(sequence_length, prev_inputs, lfu, lru, cache, preprocess_func, cache_size):
    assert prev_inputs.shape[SEQ_DIM] == sequence_length
    current_input = get_single_length_input(lfu, lru, cache, preprocess_func, cache_size)
    return np.vstack((prev_inputs[1:], current_input[None]))
    

def get_output(pre_cache, post_cache):
    pass

# Taken from Shehbaz.
def getY(C,D):
    assert(len(C) == len(D))
    Y_current = []
    KV_sorted = Counter(D)
    evict_dict = dict(KV_sorted.most_common(eviction))
    assert(len(evict_dict) == eviction)
    all_vals = evict_dict.values()
    for e in C:
        if e in evict_dict.values():
            Y_current.append(1)
        else:
            Y_current.append(0)
    #print (Y_current.count(1))
    assert(Y_current.count(1) == eviction)
    assert((set(all_vals)).issubset(set(C)))
    return Y_current

def populateData(sequence_length, sequence_data, LFUDict, LRUQ, C, D, cache_size):
#def populateData(LFUDict, LRUQ, C, D, CacheTS, CachePID):
    global X,Y
    C = list(C)
    Y_current = getY(C, D)
    #X_current = getX(LRUQ, LFUDict, C, CacheTS, CachePID)
    X_current = get_multiple_length_input(sequence_length, sequence_data, LFUDict, LRUQ, C, lambda x: x, cache_size)

    Y = np.vstack((Y, np.array(Y_current)[None]))
    X = np.vstack((X, X_current[None]))

    assert(Y_current.count(1) == eviction)
    return Y_current, X_current

### Belady Optimal Algorithm (From Shehbaz)

In [16]:
def belady_opt(blocktrace, frame):
    global maxpos, sequence_data, sequence_length
    
    optimal = defaultdict(deque)
    deleted = defaultdict(int)
    lfu = defaultdict(int)
    lru = []

    # Build the whole index for finding optimal eviction ordering
    for request_time, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        optimal[block].append(request_time)

    hit, miss = 0, 0

    cache = []
    
    for request_time, block in enumerate(tqdm(blocktrace, desc="OPT")):
        # increase frequency count
        lfu[block] +=1

        # Remove the block i at time step j from the index
        if len(optimal[block]) is not 0 and optimal[block][0] == request_time:
            optimal[block].popleft()

        
        if block in cache:
            # Cache Hit
            # Update block to MRU position
            hit += 1
            lru.remove(block)
            lru.append(block)
            
            assert request_time in deleted
            
            del deleted[request_time]
            if len(optimal[block]) is not 0:
                deleted[optimal[block][0]] = block
                optimal[block].popleft()
            else:
                deleted[maxpos] = block
                maxpos -= 1
            sequence_data = get_multiple_length_input(sequence_length, sequence_data, lfu, lru, cache, lambda x: x, cache_size)
        else:
            # Cache Miss
            miss += 1
            
            if len(cache) == frame:
                # Cache is full
                assert(len(deleted) == frame)
                evictpos = max(deleted)
                
                #if (seq_number % sampling_freq +1 == sampling_freq):
                y_opt, sequence_data = populateData(sequence_length, sequence_data, lfu, lru, cache, deleted, cache_size)
    
                cache[cache.index(deleted[evictpos])] = block
                lru.remove(deleted[evictpos])
                del deleted[evictpos]
            else:
                # Cache isn't full
                cache.append(block)
                sequence_data = get_multiple_length_input(sequence_length, sequence_data, lfu, lru, cache, lambda x: x, cache_size)
            
            # Add the candidate victim page to the j'th time step
            if len(optimal[block]) is not 0:
                deleted[optimal[block][0]] = block
                optimal[block].popleft()
            else:
                deleted[maxpos] = block
                maxpos -= 1
            lru.append(block)


    hitrate = hit / (hit + miss)

    return hitrate

## Generate training data

In [34]:
X = np.array([], dtype=np.int64).reshape(0, sequence_length, cache_size, num_params)
Y = np.array([], dtype=np.int64).reshape(0, cache_size)
trainHitrate = belady_opt(trainBlockTrace, cache_size)
X_train = X
Y_train = Y

training_data_file = 'fiu_01.pkl'
with open(training_data_file, 'wb+') as f:
    pickle.dump([X, Y], f)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
15
15
15
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
(0, 5, 100, 3) (5, 100, 3)
(1, 5, 100, 3) (5, 100, 3)
(2, 5, 100, 3) (5, 100, 3)
(3, 5, 100, 3) (5, 100, 3)
(4, 5, 100, 3) (5, 100, 3)
(5, 5, 100, 3) (5, 100, 3)
(6, 5, 100, 3) (5, 100, 3)



In [ ]:
with open(training_data_file, 'rb') as f:
    x, y = pickle.load(f)
    print(x.shape)
    test = set()
    for i in x[0,:,2]:
        test.add(i)
    for i in x[1,:,2]:
        test.add(1)
    print(test == test2)
    print(len(test))
    print(x[0:2,:,2])


#### Sample test

In [ ]:
lfu = {2: 1, 3: 2, 4: 1}
lru = [2, 3, 4]
cache = [3, 2, 4]
hidden = np.zeros((5, 3, 3))

hidden = get_multiple_length_input(5, hidden, lfu, lru, cache, lambda x: x)
print(hidden)
hidden = get_multiple_length_input(5, hidden, lfu, lru, cache, normalize_columns)
print(hidden)

In [ ]:
blocktrace = trainBlockTrace
OPT = defaultdict(deque)
for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block].append(i)
OPT